In [3]:
import pandas as pd

# Read Parquet file
df = pd.read_parquet('../VideoEval-Pro/data/test-00000-of-00001.parquet')  # Uses pyarrow by default if available
df.to_json('./output.json', orient='records', lines=True)

In [24]:
import json

with open("./output.json", "r") as f:
    dataset = f.readlines()

list_of_json = []
video_ids = []
for data in dataset:
    data = json.loads(data)

    options = data.pop("options")
    options = [option.split(" ")[1] for option in options]
    del data["answer"]
    video_id = data.pop("video")

    if video_id in video_ids:
        tmp = [json for json in list_of_json if json["video_id"] == video_id][0]
        tmp["conversations"].append({
            "question": data.pop("question"),
            "choices": options,
            "answer": data.pop("answer_text"),
            "meta": data.pop("meta"),
            "qa_subtype": data.pop("qa_subtype"),
            "qa_type": data.pop("qa_type"),
        })
    else:
        video_ids.append(video_id)

        data["video_id"] = video_id
        data["video_path"] = f"data/videoeval_pro/videos/{video_id}"
        data["conversations"] = [{
            "question": data.pop("question"),
            "choices": options,
            "answer": data.pop("answer_text"),
            "meta": data.pop("meta"),
            "qa_subtype": data.pop("qa_subtype"),
            "qa_type": data.pop("qa_type"),
        }]
        list_of_json.append(data) 

print(len(list_of_json))

with open("./test.json", "w") as f:
    json.dump(list_of_json, f, indent=4)


465
